In [1]:
# ! pip install alibi-detect
# ! pip install alibi
# ! pip install snowflake-connector-python
# ! pip install snowflake-sqlalchemy
# ! pip install xgboost

In [1]:
from utils import data_drift_monitor,model_drift_monitor,retrain_model,finalize_model,deploy_model
import warnings
warnings.filterwarnings('ignore')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


### Monitoring, Retraining and Deployment Pipeline

In [2]:
# checking the data drfit condition
data_drift_log =  data_drift_monitor(0)
data_drift = data_drift_log['is_Drift'].sum()>0

# checking for the model drift condition
labels=[False,True]
model_drift_log= model_drift_monitor(0)
model_drift = labels[model_drift_log['Model Drift ']]

# getting the cutoff_date
cut_off_date = model_drift_log['Time-Period'].split(' ')[2]

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


There is no Model drift.


#### Retraining Trigger 

In [3]:
cut_off_date

'2022-12-02'

In [4]:
# retraining the model if either model or data drift occurs
if data_drift or model_drift:
    
    # perform retraining
    new_model_metrics,old_model_metrics = retrain_model(cut_off_date)
    
    # finzaling which model to deploy based on the performance metrics
    selected_model = finalize_model(new_model_metrics,old_model_metrics)
    
    print(f"{selected_model} is selected")
    
    # deploying the selected model
    deploy_model(selected_model)

Successfully fetched data from snowflake

Shape of fetched data is  (243790, 25)
Train, Test split done

Preprocessing of Train and test data is done

Feature selection executed

Test performance of new retrained model 

RMSE is 11.950087054359967
MAE is 8.59651992861392


Successfully saved the retrained model

Sucessfully loaded old model
Test performance of old existing model
RMSE is 11.859149460941214
MAE is 8.501115407495538
New Model  is selected
Deployment New Model Successfully
